# GP Noise Test
This notebook simulates a light curve using `SimulateLightCurve`, fits Gaussian Processes with and without errors,
and compares the inferred white noise level to the PSD flattening and Poisson stats.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from stela_toolkit.data_simulator import SimulateLightCurve
from stela_toolkit.gaussian_process import GaussianProcess
from stela_toolkit.power_spectrum import PowerSpectrum

In [ ]:
dt = 10.0
n_points = 1000
time_grid = np.arange(0, n_points * dt, dt)

lc_with_errors = SimulateLightCurve(
    time_grid=time_grid,
    psd_type="powerlaw",
    psd_params={"slope": 2.0, "plnorm": 1.0},
    mean=10.0,
    std=2.0,
    add_noise=True,
)

lc_without_errors = SimulateLightCurve(
    time_grid=time_grid,
    psd_type="powerlaw",
    psd_params={"slope": 2.0, "plnorm": 1.0},
    mean=10.0,
    std=2.0,
    add_noise=False
)

In [ ]:
# fit GP with errors (FixedNoiseLikelihood)
gp_with_errors = GaussianProcess(
    lightcurve=lc_with_errors.simlc,
    kernel_form='RBF',
    run_training=True,
    num_iter=1000,
    learn_rate=0.01,
    white_noise=True,
    verbose=False
)

In [ ]:
# fit GP without errors (GaussianLikelihood)
gp_without_errors = GaussianProcess(
    lightcurve=lc_without_errors.simlc,
    kernel_form='RBF',
    run_training=True,
    num_iter=1000,
    learn_rate=0.01,
    white_noise=True,
    verbose=False
)

/home/clewin/.local/lib/python3.12/site-packages/gpytorch/likelihoods/noise_models.py:150: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 0.0001.
  warnings.warn(


In [ ]:
# compute power spectra, need to renormalize to absolute units, not RMS norm (as in code)
dt = lc_with_errors.simlc.times[1] - lc_with_errors.simlc.times[0]
n = len(lc_with_errors.simlc.rates)

psd_with = PowerSpectrum(lightcurve_or_model=lc_with_errors.simlc, num_bins=10, bin_type='log', norm=False)
psd_with_abs = psd_with.powers / (n / (2 * dt))

psd_without = PowerSpectrum(lightcurve_or_model=lc_without_errors.simlc, num_bins=10, bin_type='log', norm=False)
psd_without_abs = psd_without.powers / (n / (2 * dt))

In [ ]:
mu = lc_with_errors.simlc.mean
errors = lc_with_errors.simlc.errors

# GP params
param_errors = gp_with_errors.get_hyperparameters()
extra_noise = param_errors['likelihood.second_noise_covar.noise']
param_noerrors = gp_without_errors.get_hyperparameters()
learned_gp_noise = param_noerrors['likelihood.second_noise_covar.noise']

# noise estimate (flattening level, and from errors)
psd_flat_level = np.mean(psd_with_abs[-3:])  # adjust bin count as needed
expected_noise_from_psd = psd_flat_level
expected_noise_from_errors = np.mean(errors**2)

rescaled_gp_noise = learned_gp_noise * mu**2
rescaled_extra_noise = extra_noise * mu**2

print("\n=== GP Noise Comparison (Using Manual PSD Normalization):===")
print(f"Sampling interval (dt):         {dt:.3f}")
print(f"Mean rate (mu):                 {mu:.3f}")
print(f"PSD flattening level:           {psd_flat_level:.4e}")
print(f"Implied noise from PSD:         {expected_noise_from_psd:.4e}")
print(f"Mean(errors^2):                 {expected_noise_from_errors:.4e}")
print(f"GP noise (no errors, scaled):   {rescaled_gp_noise:.4e}")
print(f"GP noise (with errors):   rescaled_extra_noise:.4e}")


=== GP Noise Comparison (Using Manual PSD Normalization):===
- Sampling interval (dt):        10.000
Mean rate (mu):                9.932
PSD flattening level:          4.8468e+01
Implied noise from PSD:        4.8468e+01
Mean(errors^2):                9.9319e-01
GP noise (no errors, scaled):  5.9715e+01
GP extra noise (with errors):  4.9244e+01
